!pip install git+https://github.com/openai/CLIP.git
!python -m spacy download en_core_web_md

In [1]:
from PIL import Image, ImageDraw
import clip 
import json 
import numpy as np 
import torch 
from torchvision.ops import nms
from torch.utils.data import Dataset, DataLoader, SequentialSampler
import cv2 
from itertools import chain, combinations
# import pandas as pd 
# import spacy
# import en_core_web_md
# nlp = en_core_web_md.load()
import torch.nn.functional as F
import random
from tqdm import tqdm 


In [2]:
a = torch.load('/home/saaket/embodiedAI/lsd/data/processed/conventional/best_bbox_arr_valUnseen_one_utterance_all_floors.pt')

In [3]:
len(a)

579

In [7]:
a[0]

{'episodeId': '1',
 'detailedNavPath': [['471a6f3beedb4cc7a71edc7fc1c5275b'],
  ['471a6f3beedb4cc7a71edc7fc1c5275b'],
  ['471a6f3beedb4cc7a71edc7fc1c5275b']],
 'dialogArray': ['Hey what kind of room are you in?',
  'im in an office with a large wooden desk, theres a standing globe and wooden floor.',
  'is there a rug on the ground?',
  'yes there is a rug under the desk and desk chairs.',
  'where in the room are you standing.',
  'in the corner of the room near the door and far from the windows.'],
 'finalLocation': {'floor': 1,
  'mesh_coord': [7.86318, 5.2728],
  'pixel_coord': [854, 167],
  'viewPoint': '471a6f3beedb4cc7a71edc7fc1c5275b'},
 'navPath': ['471a6f3beedb4cc7a71edc7fc1c5275b'],
 'scanName': 'QUCTc6BB5sX',
 'socketId': 'ibBicE-vFhtqpClAAAD4-cuDubhJu4ZLnC2iAAD3',
 'processed_dialog_array': ['im in an office with a large wooden desk, theres a standing globe and wooden floor.',
  'yes there is a rug under the desk and desk chairs.',
  'in the corner of the room near the doo

In [4]:
b = torch.load('/home/saaket/embodiedAI/lsd/data/processed/conventional/best_bbox_arr_valSeen_one_utterance_all_floors.pt')

In [6]:
len(b)

305

In [4]:
import numpy as np 
scan_bbox_arr = np.load('/home/saaket/embodiedAI/lsd/data/interim/conventional/scan_bbox_valUnseen.npy', allow_pickle=True)

In [14]:
sim_score = torch.load('/home/saaket/embodiedAI/lsd/data/interim/conventional/similarity_scores_arr_valUnseen_combined.pt')

In [15]:
sim_score[0]

{'episodeId': '1',
 'detailedNavPath': [['471a6f3beedb4cc7a71edc7fc1c5275b'],
  ['471a6f3beedb4cc7a71edc7fc1c5275b'],
  ['471a6f3beedb4cc7a71edc7fc1c5275b']],
 'dialogArray': ['Hey what kind of room are you in?',
  'im in an office with a large wooden desk, theres a standing globe and wooden floor.',
  'is there a rug on the ground?',
  'yes there is a rug under the desk and desk chairs.',
  'where in the room are you standing.',
  'in the corner of the room near the door and far from the windows.'],
 'finalLocation': {'floor': 1,
  'mesh_coord': [7.86318, 5.2728],
  'pixel_coord': [854, 167],
  'viewPoint': '471a6f3beedb4cc7a71edc7fc1c5275b'},
 'navPath': ['471a6f3beedb4cc7a71edc7fc1c5275b'],
 'scanName': 'QUCTc6BB5sX',
 'socketId': 'ibBicE-vFhtqpClAAAD4-cuDubhJu4ZLnC2iAAD3',
 'processed_dialog_array': ['im in an office with a large wooden desk, theres a standing globe and wooden floor. yes there is a rug under the desk and desk chairs. in the corner of the room near the door and far 

In [10]:
def compute_image_features(scans, scan_bbox_arr, floorplan_path, model, transform):
    features = []
    for i, scan in enumerate(tqdm(scans)):
        features.append({'scanName': scan, 'features': {}})
        for level in scan_bbox_arr[i].keys():
            image_path = f'{floorplan_path}/floor_{int(level)}/{scan}_{int(level)}.png'
            image = Image.open(image_path)
            coords = scan_bbox_arr[i][level]
            anchor_dataset = AnchorImageDataset(image, coords, transform)
            anchor_loader = DataLoader(anchor_dataset, 
                    batch_size=15, 
                    sampler=SequentialSampler(anchor_dataset),
                    pin_memory=False,
                    drop_last=False,
                    num_workers=2,)
            
            anchor_features = compute_anchor_features(model, anchor_loader)
            features[i]['features'][level] = anchor_features
    return features 

In [ ]:
class RegionProposalNetwork:
    def get_coords_and_masks(self, pil_img, B=0.15, K_max_box_w=0.9, K_max_box_h=0.9,
                                K_min_box_w=0.03, K_min_box_h=0.03, iou_threshold=0.9):
        img = pil_img.copy()
        img = np.array(img)
        img = self._c_mean_shift(img)
        img = self._split_gray_img(img, n_labels=9)
        coords, masks = self._get_mixed_boxes_and_masks(
            img, B, K_max_box_w, K_max_box_h, K_min_box_w, K_min_box_h, iou_threshold
        )
        return coords, masks


    def _get_mixed_boxes_and_masks(self, image, B, K_max_box_w, K_max_box_h, K_min_box_w, K_min_box_h, iou_threshold):
        img = image.copy()
        h, w = img.shape
        max_box_w, max_box_h, min_box_w, min_box_h = w * K_max_box_w, h * K_max_box_h, w * K_min_box_w, h * K_min_box_h

        out_boxes = []
        out_masks = []

        labels = np.unique(img)
        combs = self._get_combinations(labels)

        comb_indexes = []
        for i, comb in enumerate(combs):
            n_img = np.isin(img, np.array(comb)).astype(np.uint8) * 255
            n_img = self._clear_noise(n_img)
            m_boxes = self._get_boxes_from_mask(n_img, max_box_w, max_box_h, min_box_w, min_box_h)
            out_boxes.extend(m_boxes)
            comb_indexes.extend([i] * len(m_boxes))

        comb_indexes = np.array(comb_indexes)

        boxes = torch.tensor(out_boxes, dtype=torch.float32)
        labels = torch.ones(boxes.shape[0], dtype=torch.float32)

        indexes = nms(boxes, labels, iou_threshold)

        out_boxes = boxes[indexes].numpy().astype(np.int32)
        comb_indexes = comb_indexes[indexes.numpy()]

        for (x1, y1, x2, y2), comb_index in zip(out_boxes, comb_indexes):
            comb = combs[comb_index]
            n_img = np.isin(img, np.array(comb)).astype(np.uint8) * 255
            n_img = self._clear_noise(n_img)
            mask = n_img[y1:y2, x1:x2]

            h, w = mask.shape
            h_b = int(h * B)
            w_b = int(w * B)

            mask = mask.astype(np.bool)
            if mask[:h_b, :].sum() + mask[-h_b:, :].sum() + mask[:, :w_b].sum() + \
                    mask[:, -w_b:].sum() > 4 * h * w * B * (1 - B) * 0.5:
                mask = ~mask

            mask = mask.astype(np.uint8) * 255
            kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
            mask = cv2.morphologyEx(mask, cv2.MORPH_ERODE, kernel)
            mask = cv2.dilate(mask, kernel, iterations=3)
            mask = mask.astype(np.bool)
            out_masks.append(mask)

        return out_boxes, out_masks
    
    def _c_mean_shift(self, image):
        img = image.copy()
        img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        img = cv2.pyrMeanShiftFiltering(img, 16, 48)
        img = cv2.cvtColor(img, cv2.COLOR_HSV2RGB)
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        return img.astype(np.uint8)

    def _split_gray_img(self, image, n_labels=9):
        img = image.copy()
        step = 255 // n_labels
        t = list(np.arange(0, 255, step)) + [255]
        for i, (t1, t2) in enumerate(zip(t[:-1], t[1:])):
            img[(img >= t1) & (img < t2)] = t1
        return img

    @staticmethod
    def _get_combinations(array):
        combs = list(chain(*map(lambda x: combinations(array, x), range(0, len(array)+1))))
        return combs[1:]

    @staticmethod
    def _get_boxes_from_mask(mask, max_box_w, max_box_h, min_box_w, min_box_h):
        boxes = []
        contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        bboxes = [cv2.boundingRect(c) for c in contours]
        for i, (x, y, w, h) in enumerate(bboxes):
            if (w < max_box_w and h < max_box_h) and (w > min_box_w and h > min_box_h):
                boxes.append([x, y, x + w, y + h])
        return boxes

    @staticmethod
    def _clear_noise(image):
        img = image.copy()
        e_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
        erose = cv2.morphologyEx(img, cv2.MORPH_ERODE, e_kernel)
        d_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
        dilate = cv2.morphologyEx(erose, cv2.MORPH_DILATE, d_kernel)
        return dilate

    @staticmethod
    def _get_nearest_box_and_mask(box, gt_boxes, gt_masks):
        return sorted(zip(gt_boxes, gt_masks), key=lambda x: sum([abs(x[0][i] - box[i]) for i in range(4)]))[0]

In [8]:
class AnchorImageDataset(Dataset):
    def __init__(self, image, coords, transforms):
        self.image = image.copy()
        self.coords = coords
        self.transforms = transforms

    def __len__(self):
        return len(self.coords)

    def __getitem__(self, idx):
        coord = self.coords[idx]
        return self.transforms(self.image.crop(coord))

In [11]:
def get_unique_scans(data_file):
    with open(data_file) as j:
        samples = json.load(j)
    unique_scans = np.empty(len(samples), dtype='<U16')
    for i, sample in enumerate(samples):
        unique_scans[i] = sample['scanName']
    return np.unique(unique_scans)

In [12]:
unique_scans_val_unseen = get_unique_scans('../input/way-led/way_splits/valUnseen_data.json')

FileNotFoundError: [Errno 2] No such file or directory: '../input/way-led/way_splits/valUnseen_data.json'

In [9]:
def compute_anchor_features(model, anchor_loader):
    anchor_features = []
    for anchor_batch in anchor_loader:
        with torch.no_grad():
            anchor_features_ = model.encode_image(anchor_batch.to('cuda:0'))
        anchor_features.append(anchor_features_)
    
    anchor_features = torch.vstack(anchor_features)
    anchor_features /= anchor_features.norm(dim=-1, keepdim=True)
    return anchor_features

In [ ]:
def compute_all_bounding_boxes(unique_scan_names, scans_levels_file, rpn):

    samples = unique_scan_names
    scan_levels_file = scans_levels_file 
    with open(scan_levels_file) as j:
        scan_levels = json.load(j)
    coords_arr = []
    for i, sample in enumerate(tqdm(samples)):
        num_levels = int(scan_levels[str(sample)]["levels"])
        coords_arr.append({'scan_name': sample, 'coords': {}})
        for level in range(num_levels):
            image_path = f'../input/way-led/floorplans/floor_{level}/{sample}_{level}.png'
            image = Image.open(image_path)
            coords, masks = rpn.get_coords_and_masks(image)
            coords_arr[i]['coords'][str(level)] = coords

    return coords_arr

In [ ]:
def compute_text_features(data_file, model, mode="combined"):
    '''computes CLIP representation from dialog array. 
        for the mode argument: 
        1. combined: joins all observer utterances into one string
        2. one_utterance: considers all observer utterances separately
        3. one_sentence: considers all sentences from each observer utterance separately'''
    with open(data_file) as f:
        data = json.load(f)
    text_features = []
    text_features_arr = []
    for i, sample in enumerate(tqdm(data)):
        
        # Get the dialog array from the json dict 
        dialogs = sample["dialogArray"]
        
        # Isolate observer dialogs (dialogs in the form of: L0, O0, L1, O1 ..)
        dialogs = dialogs[1:len(dialogs):2]
        
        # Format dialog array into dialogs as specified by the mode
        if mode == 'combined':
            dialogs = [" ".join(dialogs).lower()]
        elif mode =='one_utterance':
            dialogs = dialogs 
        elif mode =='one_sentence':
            sentences = []
            for dialog in dialogs:
                sentences.extend(dialog.split("."))
            dialogs = [s.lower().strip() for s in sentences if s]
            
        tokens = clip.tokenize(dialogs, truncate=True)
#         text_features_arr.append({'scan_name': sample['scanName'], 'episode_id': sample['episodeId'], 'dialogs': dialogs})
        sample['processed_dialog_array'] = dialogs
        with torch.no_grad():
            text_features = model.encode_text(tokens.to(device))
            text_features /= text_features.norm(dim=-1, keepdim=True)
        sample['text_features'] = text_features
    return data 

In [ ]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
model, transform = clip.load("RN50", device=device)

In [ ]:
text_features = compute_text_features('../input/way-led/way_splits/valUnseen_data.json', model, mode='one_sentence')

In [ ]:
text_features[0]

In [ ]:
torch.save(text_features, 'normalized_text_features_valUnseen_combined.pt')

In [ ]:
def compute_image_features(scans, scan_bbox_arr, model, transform):
    features = []
    for i, scan in enumerate(tqdm(scans)):
        features.append({'scanName': scan, 'features': {}})
        for level in scan_bbox_arr[i].keys():
            image_path = f'../input/way-led/floorplans/floor_{int(level)}/{scan}_{int(level)}.png'
            image = Image.open(image_path)
            coords = scan_bbox_arr[i][level]
            anchor_dataset = AnchorImageDataset(image, coords, transform)
            anchor_loader = DataLoader(anchor_dataset, 
                    batch_size=15, 
                    sampler=SequentialSampler(anchor_dataset),
                    pin_memory=False,
                    drop_last=False,
                    num_workers=2,)
            
            anchor_features = compute_anchor_features(model, anchor_loader)
            features[i]['features'][level] = anchor_features
    return features 

In [ ]:
# rpn = RegionProposalNetwork()
# dataset_path = '../input/way-led'
# # Load CLIP model 

# image_features = compute_image_features()


# data_file = f'{dataset_path}/way_splits/valUnseen_data.json'
# scan_levels_file = f'{dataset_path}/floorplans/scan_levels.json'

# unique_scans_val_unseen = get_unique_scans(data_file)
# sample_with_bboxes = compute_all_bounding_boxes(unique_scans_val_unseen, scan_levels_file, rpn)
# image_features = compute_image_features(unique_scans_val_unseen, scan_bbox_arr, model, transform)

In [ ]:
# torch.save('image_features', image_features)
image_features = torch.load('../input/rpn-ccv-led/RPN_CCV/normalized_features.pt')

In [ ]:
def compute_scan2idx(unique_scans):
    scan2idx = {}
    for i, scan in enumerate(unique_scans):
        scan2idx[scan] = i
    return scan2idx

In [ ]:
scan2idx = compute_scan2idx(unique_scans_val_unseen)
# All features are saved in an array. scand2idx correspond to index of array

In [ ]:
bbox_arr = np.load('../input/rpn-ccv-led/RPN_CCV/scan_bbox_valUnseen.npy', allow_pickle=True)

In [ ]:
bbox_arr[0]

In [ ]:
def get_similarity_scores(image_features_arr, text_features_arr, scan2idx):
    similarity_scores_arr = []
    for i, sample in enumerate(tqdm(text_features_arr)):
        text_features = sample['text_features']
        correct_scan = image_features_arr[scan2idx[sample['scanName']]]
        max_len = 0
        for floor in correct_scan['features'].keys():
            if len(correct_scan['features'][floor]) > max_len:
                max_len = len(correct_scan['features'][floor])
        similarity_scores = []
        for image_features in correct_scan['features'].values():
            similarity = (text_features @ image_features.T)
            similarity = F.pad(similarity, pad=(0, max_len - similarity.size(-1)), value=float('-inf'))
            similarity_scores.append(similarity)
        similarity_scores = torch.hstack(similarity_scores)
        sample['similarity_scores'] = similarity_scores
        sample['max_len'] = max_len
    return text_features_arr

In [ ]:
similarity_scores_arr = get_similarity_scores(image_features, text_features, scan2idx)

In [ ]:
torch.save(similarity_scores_arr, 'similarity_scores_arr_combined.pt')

In [ ]:
def get_best_bbox(similarity_scores_arr, bbox_arr, scan2idx, mode='all_floors'):
    
    best_bboxes = []
    if mode == 'all_floors':
        for sample in tqdm(similarity_scores_arr):
            best_idx = list(sample['similarity_scores'].detach().cpu().argmax(dim=-1))
            best_idx = [idx.item() for idx in best_idx]
            best_floors = [idx // sample['max_len'] for idx in best_idx]
            best_boxes = [idx % sample['max_len'] for idx in best_idx]
            dialogs = sample['processed_dialog_array']
            scan_idx = scan2idx[sample['scanName']]
            coords = bbox_arr[scan_idx]
            sample['floors'] = {}
            for i, floor in enumerate(best_floors):
                if floor not in sample['floors']:
                    sample['floors'].update({floor: []})
                sample['floors'][floor].append((dialogs[i], coords[str(floor)][best_boxes[i]]))
            sample['bboxes'] = best_boxes
            sample['floors']
#     if mode == 'correct_floor':
#         for sample in tqdm(similarity_scores_arr):
#             correct_floor = sample['finalLocation']['floor']
#             best_boxes = list(sample['similarity_scores'].detach().cpu()[:, sample['max_len']*correct_floor:sample['max_len']*(correct_floor+1)].argmax(dim=-1))
#             sample['floors'] = {correct_floor: []}
#             for i, idx in enumerate(best_boxes):
#                 sample['floors'][correct_floor].append((dialogs[i], coords[str(correct_floor)][best_boxes[i]]))
            
    return similarity_scores_arr 
        

In [ ]:
best_bbox_arr = get_best_bbox(similarity_scores_arr, bbox_arr, scan2idx, mode='all_floors')

In [ ]:
torch.save(best_bbox_arr, 'best_bbox_arr_combined_all_floors.pt')

In [ ]:
def visualize_bboxes(best_bbox_arr, example_num, idx):
    bbox_dict = best_bbox_arr[example_num]
    with open('../input/way-led/way_splits/valUnseen_data.json') as jj:
        valUnseen_data = json.load(jj)
    x, y = valUnseen_data[example_num]['finalLocation']['pixel_coord'] 
    ideal_floor = valUnseen_data[example_num]['finalLocation']['floor'] 
    
    floors = bbox_dict['floors']
    unique_floors = list(set(floors.keys()))
    try:
        viz_floor = unique_floors[idx]
    except:
        print("This floor does not include a good bounding box")
        return 
    image = Image.open(f'../input/way-led/floorplans/floor_{viz_floor}/{bbox_dict["scanName"]}_{viz_floor}.png')
    for utterance, bbox in floors[viz_floor]:
        x1, y1, x2, y2 = bbox
        draw = ImageDraw.Draw(image)
        r = random.randint(0,255)
        g = random.randint(0,255)
        b = random.randint(0,255)
        rgb = (r,g,b)
        draw.rectangle((x1, y1, x2, y2), width=2, outline=rgb)
        draw.text((x1, y1), utterance, fill=(r, g, b, 128))
    if viz_floor == ideal_floor:
        r = 10
        draw.ellipse([(x-r, y-r), (x+r, y+r)], fill = 'red', outline ='red')
    return (image, bbox_dict['floors'][viz_floor], viz_floor,bbox_dict['scanName'])

In [ ]:
image, best_bbox, bbox_floor, scan_name = visualize_bboxes(best_bbox_arr, 0, 0)

In [ ]:
image

In [ ]:
bbox_floor

In [ ]:
scan_name

In [ ]:
def inside_bbox(x, y, bbox):
    # Check if a point lies fully inside a bbox
    # bbox [x1, y1, x2, y2]
    # x increases to the right, y increases downwards
    x_left, y_top, x_right, y_bottom = bbox
    if x >= x_left and x <= x_right and y>=y_top and y<=y_bottom:
        return True
    else:
        return False

In [ ]:

def closest_in_scan(cx, cy, bbox_floor, node2pix, scan_name):
    closest_vp = None
    closest_dist = float('inf')
    c_point = np.array([cx, cy])
    closest_point = 0, 0
    for viewpoint in node2pix[scan_name]:
        pixels, real, floor = node2pix[scan_name][viewpoint]
        if floor == bbox_floor:
            v_point = np.array(pixels)
            dist = np.linalg.norm(c_point - v_point)
            if dist <= closest_dist:
                closest_vp = viewpoint
                closest_dist = dist
                closest_point = tuple(pixels)
    return closest_vp, closest_point
        
        

In [ ]:
def get_viewpoints_inside_bbox(bbox, bbox_floor, node2pix, scan_name):
    viewpoints = node2pix[scan_name]
    points_in_bbox = []
    coordinates_in_bbox = []
    for viewpoint in viewpoints:
        v_x, v_y = node2pix[scan_name][viewpoint][0]
        vp_floor = node2pix[scan_name][viewpoint][-1]
        if vp_floor == bbox_floor and inside_bbox(v_x, v_y, bbox):
            points_in_bbox.append(viewpoint)
            coordinates_in_bbox.append((v_x, v_y))
    if len(points_in_bbox) == 0:
        cx, cy = ( bbox[0] + bbox[2] ) // 2 , ( bbox[1] + bbox[3] ) // 2
        approx_viewpoint, approx_coords = closest_in_scan(cx, cy, bbox_floor, node2pix, scan_name)
        points_in_bbox.append(approx_viewpoint)
        coordinates_in_bbox.append(approx_coords)
    return points_in_bbox, coordinates_in_bbox
            
        
    

In [ ]:
empties

In [ ]:
def get_closest_viewpoint_distance(viewpoints, gt_viewpoint, node2pix, geodistance_nodes, scan_name):
    closest_distance = float('inf')
    closest_vp = None
    for v in viewpoints:
        dist = geodistance_nodes[scan_name][gt_viewpoint][v]
        if dist < closest_distance:
            closest_distance = dist
            closest_vp = v
    return closest_distance, closest_vp

In [ ]:
# Test 
allscans_node2pix = json.load(open('../input/way-led/floorplans/allScans_Node2pix.json'))
geodistance_nodes = json.load(open('../input/way-led/geodistance_nodes.json'))

In [ ]:
points_in_bbox, coordinates_in_bbox = get_viewpoints_inside_bbox(best_bbox[0][-1], bbox_floor, allscans_node2pix, scan_name)

In [ ]:
for (x, y) in coordinates_in_bbox:
    draw = ImageDraw.Draw(image)
    r = 2
    draw.ellipse([(x-r, y-r), (x+r, y+r)], fill = 'red', outline ='red')

In [ ]:
image

In [ ]:
# Process 
# Have floors and best boxes for each floor 
# For all floor box combinations obtain the viewpoints in these bboxes, 
# 

In [ ]:
def compute_metrics(best_bbox_arr):
    # 1. Average distance to closest viewpoint that is inside a bounding box or closest to the center of the bounding box if not point in bbox
    allscans_node2pix = json.load(open('../input/way-led/floorplans/allScans_Node2pix.json'))
    geodistance_nodes = json.load(open('../input/way-led/geodistance_nodes.json'))
    # Combined 
    empties = []
    dists = []
    failed = []

    for candidate in best_bbox_arr:

        floors = list(candidate['floors'].keys())
        view_points_in_all_bboxes = []
        for floor in floors:
            for item in candidate['floors'][floor]:
                dialog, bbox = item
                view_points_in_bbox, _ = get_viewpoints_inside_bbox(bbox, floor, allscans_node2pix, candidate['scanName'])
                view_points_in_all_bboxes.extend(view_points_in_bbox)
#         print(view_points_in_all_bboxes, candidate['finalLocation']['viewPoint'], candidate['scanName'])
        try:
            closest_dist, _ = get_closest_viewpoint_distance(view_points_in_all_bboxes, candidate['finalLocation']['viewPoint'], allscans_node2pix, geodistance_nodes, candidate['scanName'])
            dists.append(closest_dist)
        except:
            failed.append((view_points_in_all_bboxes, candidate['finalLocation']['viewPoint'], candidate['scanName']))
#         print('_____________________')
    return dists, failed

In [ ]:
allscans_node2pix['QUCTc6BB5sX']

# 1: 8a934837258740e686974eb02265db7f
# 0: 695637592c45427c96e587418963bd18
geodistance_nodes['QUCTc6BB5sX']['8a934837258740e686974eb02265db7f']['695637592c45427c96e587418963bd18']

In [ ]:
dist, failed = compute_metrics(best_bbox_arr)

In [ ]:
dist_np = np.array(dist)

In [ ]:
dist_np

In [ ]:
print((dist_np <= 1).sum()), print((dist_np <= 3).sum()), print((dist_np <= 5).sum()), print(len(dist))

In [ ]:
print((dist_np <= 1).sum()), print((dist_np <= 3).sum()), print((dist_np <= 5).sum()), print(len(dist))

In [ ]:
print((dist_np <= 1).sum()), print((dist_np <= 3).sum()), print((dist_np <= 5).sum()), print(len(dist))

In [ ]:
plt.hist(dist_np, bins = list(range(0, 30)))

In [ ]:
dist_np = np.array(dist)

In [ ]:
print((dist_np <= 3).sum()), print(len(dist))

In [ ]:
from collections import Counter 


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.hist(dist, bins = list(range(0, 30)))

In [ ]:
dist

In [ ]:
# 1c18c47760ef40c394ac52df1d036047
# 40d9295b108e4736a7c1e7dc5e83feec

In [ ]:
geodistance_nodes['oLBMNvg9in8']['1c18c47760ef40c394ac52df1d036047']['40d9295b108e4736a7c1e7dc5e83feec']

In [ ]:
allscans_node2pix['oLBMNvg9in8']

In [ ]:
list(geodistance_nodes['oLBMNvg9in8'].keys())

In [ ]:
print("something")

In [ ]:
for item in allscans_node2pix['2azQ1b91cZZ']:
    print(item)

In [ ]:
geodistance_nodes['2azQ1b91cZZ']['a9d9d5e3d5e44d5080b643057d25fc27']

In [ ]:
data_file = '../input/way-led/way_splits/valUnseen_data.json'
scan_levels_file = '../input/way-led/floorplans/scan_levels.json' 

In [ ]:
sample_with_bboxes = compute_all_bounding_boxes(unique_scans_val_unseen, scan_levels_file, rpn)

In [ ]:
feat[0]

In [ ]:
sample_with_bboxes[0]

In [ ]:
scan_bbox_arr = [] 
for scan in sample_with_bboxes.keys():
    scan_bbox_arr.append(sample_with_bboxes[scan])

In [ ]:
np.save("scan_bbox_valUnseen.npy", scan_bbox_arr)

In [ ]:
# Format:

# ```
# arr = [
#     'scanName' : {
#         'floor': {
#             tensor of features of size (num_bboxes * 1024)
#         }, 
#         'floor2' : {
#             tensor of features of size (num_bboxes * 1024)
#         }
#     }, 
#     'scanName' : {
#         'floor': {
#             tensor of features of size (num_bboxes * 1024)
#         }
#     },
    
# ]

# ```

In [ ]:
scan_bbox_arr[0]['0'].dtype

In [ ]:
def compute_anchor_features(model, anchor_loader):
    anchor_features = []
    for anchor_batch in anchor_loader:
        with torch.no_grad():
            anchor_features_ = model.encode_image(anchor_batch.to(device))
        anchor_features.append(anchor_features_)
    
    anchor_features = torch.vstack(anchor_features)
    anchor_features /= anchor_features.norm(dim=-1, keepdim=True)
    return anchor_features

In [ ]:
feat = compute_image_features(unique_scans_val_unseen, scan_bbox_arr, model, transform)

In [ ]:
torch.save(feat, 'normalized_features.pt')

In [ ]:
features_loaded = torch.load('normalized_features.pt')

In [ ]:
torch.save(text_features, 'normalized_text_features_valUnseen.pt')

In [ ]:
def process_one_sample(training_samples, idx):
    sample = training_samples[idx]
    
    scan_levels_file = '../input/way-led/floorplans/scan_levels.json' 
    with open(scan_levels_file) as j:
        scan_levels = json.load(j)
    num_levels = int(scan_levels[sample['scanName']]["levels"])
    
    dialogs = sample["dialogArray"]
    dialogs = dialogs[1:len(dialogs):2]
    # Isolate observer dialogs
    excluded_tags = {"PRON", "X"}
#     for i, dialog in enumerate(dialogs):
#         new_dialog = []
#         for token in nlp(dialog):
#             if token.pos_ not in excluded_tags:
#                 new_dialog.append(token.text)
#         dialogs[i] = " ".join(new_dialog)
    new_dialogs = []
    for dialog in dialogs:
        new_dialogs.extend(dialog.split("."))
    dialogs = [d.lower().strip().replace("  ", " ") for d in new_dialogs if d != '']
    tokens = clip.tokenize(dialogs)
    text_features = compute_text_features(model, tokens)
        
    x, y = training_samples_list[idx]['finalLocation']['pixel_coord']
    
    max_so_far = float('-inf')
    num_boxes_per_image = []
    text_image_sim = []
    coords_by_image = []
    max_len = 1000
    best_floor_so_far = 0
    max_similarity_so_far = float('-inf')
    best_similarity_score = None
    for level in range(num_levels):
        image_path = f'../input/way-led/floorplans/floor_{level}/{sample["scanName"]}_{level}.png'
        image = Image.open(image_path)
        coords, masks = rpn.get_coords_and_masks(image)
        coords_by_image.append(coords)
        anchor_dataset = AnchorImageDataset(image, coords, transform)
        num_boxes_per_image.append(len(coords))
        anchor_loader = DataLoader(anchor_dataset, 
                batch_size=10, 
                sampler=SequentialSampler(anchor_dataset),
                pin_memory=False,
                drop_last=False,
                num_workers=2,)

        anchor_features = compute_anchor_features(model, anchor_loader)
        out = (text_features @ anchor_features.T)
#         out = F.pad(out, pad=(0, max_len - out.size(-1)), value=float('-inf'))
#         print(out.size())
        if out.sum().item() > max_similarity_so_far:
            max_similarity_so_far = out.sum().item()
            best_floor_so_far = level
            best_similarity_score = out.cpu()
    
#         text_image_sim.append(out)
#     similarity_scores = torch.hstack(text_image_sim)
#     box_idx_tensor = torch.argmax(similarity_scores, dim=-1)
#     box_idx = list(box_idx_tensor % max_len)
#     image_levels = list(box_idx_tensor // max_len)
#     images = [Image.open(f'../input/way-led/floorplans/floor_{level}/{sample["scanName"]}_{level}.png') for level in image_levels]
#     boxes = [coords_by_image[image_levels[i]][i] for i in box_idx]
#     boxes = []
#     for i in range(len(box_idx)):
#         boxes.append(coords_by_image[image_levels[i]][box_idx[i]])
    box_idx = list(torch.argmax(best_similarity_score, dim=-1))
    image = Image.open(f'../input/way-led/floorplans/floor_{best_floor_so_far}/{sample["scanName"]}_{best_floor_so_far}.png')
    boxes = coords_by_image[best_floor_so_far][box_idx]
    return boxes, dialogs, image, x, y, coords_by_image
    

In [ ]:
# Download Features 
import gdown

url = 'https://drive.google.com/drive/folders/14Isz8pGamCLnax-RgEhteXOmgcgF8j04?usp=sharing'
output = "out.zip"
gdown.download(url, output)

In [ ]:
image_features = torch.load('../input/rpn-ccv-led/RPN_CCV/normalized_features.pt')
text_features = torch.load('../input/rpn-ccv-led/RPN_CCV/normalized_text_features_valUnseen.pt')

In [ ]:
def get_best_boxes(image_features, text_features):
    
    for feature in text_features:
        

In [ ]:
boxes, dialogs, image, x, y, coords_by_images = process_one_sample(training_samples_list, 2)

In [ ]:
for i, (x1, y1, x2, y2) in enumerate(boxes):
#     img = image[i]
    draw = ImageDraw.Draw(image)
    r = random.randint(0,255)
    g = random.randint(0,255)
    b = random.randint(0,255)
    rgb = (r,g,b)
    draw.rectangle((x1, y1, x2, y2), width=2, outline=rgb)
    


# r = 10
# draw.ellipse([(x-r, y-r), (x+r, y+r)], fill = 'red', outline ='red')



In [ ]:
dialogs

In [ ]:
image

In [ ]:
image

In [ ]:
image.size

In [ ]:
image.save('random_example_3.png')

In [ ]:
dist = Image.new('RGB', (1200, 700), (0, 0, 0))
draw = ImageDraw.Draw(dist)

In [ ]:
for i, (x1, y1, x2, y2) in enumerate(boxes):
    draw.rectangle((x1, y1, x2, y2), fill=(255, 255, 255), outline=(255, 255, 255))

In [ ]:
dist.save('output_dist.png')

In [1]:
arr = ['i am standing in a living room area it looks like. the floor has a blue and white patterned design. i am standing next to a tv. in front of the tv is a little table with something orange on it, maybe flowers. in the corner is a sofa by itself, the table is in front of that sofa but i am by the tv. this room is large with a pool table in it as well. yes a red pool table in the middle of the room.']

In [3]:
dialogs = [" ".join(arr)]

In [4]:
dialogs

['i am standing in a living room area it looks like. the floor has a blue and white patterned design. i am standing next to a tv. in front of the tv is a little table with something orange on it, maybe flowers. in the corner is a sofa by itself, the table is in front of that sofa but i am by the tv. this room is large with a pool table in it as well. yes a red pool table in the middle of the room.']